In [1]:
import sys
# import pyccl as ccl
import pickle
import camb
sys.path.insert(0,'../skylens/')
# from importlib import reload

In [21]:
sys.path

['./',
 '/verafs/scratch/phy200040p/sukhdeep/project/skylens/notebooks/dask-worker-space/worker-w4edxi79',
 '../skylens/',
 '/home/sukhdees/project/code/python_scripts/',
 '/home/sukhdees/project/code/camb17/pycamb/',
 '/home/sukhdees/project/code/python_scripts/',
 '/verafs/scratch/phy200040p/sukhdeep/project/skylens/notebooks',
 '/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python38.zip',
 '/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8',
 '/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/lib-dynload',
 '',
 '/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages',
 '/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/IPython/extensions',
 '/verafs/home/sukhdeep/.ipython',
 './ForQuE/']

In [28]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
c=LocalCluster(n_workers=2,processes=False,memory_limit='25gb',threads_per_worker=4,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(c)

06:57:59 DEBUG:Using selector: EpollSelector
06:57:59 DEBUG:Using selector: EpollSelector
/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41363 instead
  warnings.warn(


In [29]:
len(client.scheduler_info()['workers'])

2

In [3]:
from skylens import *
from survey_utils import *

In [4]:
#only for python3
import importlib
reload=importlib.reload

In [5]:
zs=lsst_source_tomo_bins()
corr_ll=('shear', 'shear')

ns0:  27.0


In [6]:
wigner_files={}
# wigner_files[0]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_0_reorder.zarr'
# wigner_files[2]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_2_reorder.zarr'

wigner_files[0]= '/home/deep/data/repos/SkyLens/temp/dask_wig3j_l6500_w2100_0_reorder.zarr'
wigner_files[2]= '/home/deep/data/repos/SkyLens/temp/dask_wig3j_l3500_w2100_2_reorder.zarr'


# Power Spectrum covariance

In [7]:
#setup parameters
lmax_cl=200
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=20
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=True
bin_cl=True

SSV_cov=True
tidal_SSV_cov=False
Tri_cov=True

bin_xi=True
theta_bins=np.logspace(np.log10(1./60),1,20)

##  Test with 1 redshift bin

In [8]:
store_win=True
window_lmax=200

In [9]:
use_window=False
do_cov=True

In [10]:
nside=128

In [11]:
z0=1 #1087
zs_bin1=source_tomo_bins(zp=[z0],p_zp=np.array([1]),ns=30,use_window=use_window,nside=nside)

In [12]:
use_binned_l=False

In [13]:
a={'a':1,'b':2}

In [14]:
kk=list(a.keys())
ii=len(kk)

In [15]:
[(kk[i],kk[j]) for i in np.arange(ii) for j in np.arange(i,ii)]

[('a', 'a'), ('a', 'b'), ('b', 'b')]

In [16]:
import skylens
reload(skylens)
from skylens import *

In [17]:
store_win=True

In [18]:
SSV_cov=False
bin_cl=False
do_cov=True
Tri_cov=False

In [19]:
#use all ell
kappa0=Skylens(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       use_binned_l=use_binned_l,wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                       sparse_cov=True
                                       )

indxs:  None None
Win gen: step size 199 True
Window done


../skylens/cov_utils.py:58: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th


In [20]:
kappa0.l_bins

array([ 12,  13,  16,  18,  21,  24,  28,  33,  38,  44,  51,  59,  68,
        79,  91, 106, 122, 142, 164, 189])

In [21]:
%time cl0G=kappa0.cl_tomo() 

In [22]:
do_cov

True

In [23]:
bi=(0,0)
xx=cl0G['cov'][corr_ll+corr_ll][bi+bi].compute()

In [24]:
xx

{'final': array([[5.27087051e-16, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 7.98306903e-16, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 8.51538104e-16, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.81480199e-19, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 1.78269695e-19, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 1.75132760e-19]]),
 'G': array([[5.27087051e-16, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 7.98306903e-16, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.

In [25]:
kappa0.l.shape

(198,)

In [26]:
%time cl0=cl0G['stack'].compute()

In [44]:
cl0['cov']#.todense()

array([[5.27087051e-16, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.98306903e-16, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.51538104e-16, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00

In [28]:
do_xi=True
bin_xi=True
do_cov=False
bin_cl=True
th_min=2.5/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins*40)
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [29]:
# import wigner_transform 
# reload(wigner_transform)
# from wigner_transform import *

import binning
reload(binning)
from binning import *

In [30]:
th

array([0.04166667, 0.04190751, 0.04214975, 0.04239339, 0.04263844,
       0.0428849 , 0.04313279, 0.04338211, 0.04363287, 0.04388508,
       0.04413875, 0.04439389, 0.0446505 , 0.04490859, 0.04516818,
       0.04542926, 0.04569186, 0.04595597, 0.04622161, 0.04648879,
       0.04675751, 0.04702778, 0.04729961, 0.04757302, 0.04784801,
       0.04812458, 0.04840276, 0.04868254, 0.04896394, 0.04924697,
       0.04953163, 0.04981794, 0.0501059 , 0.05039553, 0.05068683,
       0.05097982, 0.0512745 , 0.05157088, 0.05186897, 0.05216879,
       0.05247034, 0.05277364, 0.05307869, 0.0533855 , 0.05369408,
       0.05400445, 0.05431661, 0.05463058, 0.05494636, 0.05526397,
       0.05558341, 0.0559047 , 0.05622785, 0.05655286, 0.05687976,
       0.05720854, 0.05753922, 0.05787182, 0.05820633, 0.05854278,
       0.05888118, 0.05922153, 0.05956385, 0.05990815, 0.06025443,
       0.06060272, 0.06095303, 0.06130535, 0.06165972, 0.06201613,
       0.0623746 , 0.06273515, 0.06309777, 0.0634625 , 0.06382

In [31]:
#Hankel Transform setup
WT_kwargs={'l':l0,'theta':th,'s1_s2':[(2,2),(2,-2),(0,0)]}
WT=wigner_transform(**WT_kwargs)

In [32]:
WT.wig_d.keys()

dict_keys([(2, 2), (2, -2), (0, 0)])

In [33]:
import skylens
reload(skylens)
from skylens import *

In [34]:
use_binned_theta=True

In [35]:
kappa_xi0=Skylens(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                      do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                                      use_binned_l=False,use_binned_theta=False,
                                      nz_PS=10
                                       )

In [36]:
%time xiHG0=kappa_xi0.xi_tomo()
%time xiH0=xiHG0['stack'].compute()

In [37]:
kappa_xib=Skylens(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                      do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                                      use_binned_l=use_binned_l,use_binned_theta=use_binned_theta
                                       )

In [38]:
%time xiHG_b=kappa_xib.xi_tomo()

In [39]:
%time xiH_b=xiHG_b['stack'].compute()

In [40]:
kappa_xi0.WT.wig_d.keys(),WT.wig_d.keys()

(dict_keys([(2, 2), (2, -2), (0, 0)]), dict_keys([(2, 2), (2, -2), (0, 0)]))

In [41]:
%time xiHG0=kappa_xi0.xi_tomo()
%time xiH0=xiHG0['stack'].compute()

In [42]:
xiH0['xi']

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.57974579e-06, 1.29440641e-06, 6.58178123e-07,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.86955400e-06, 2.54181368e-06, 2.43191643e-06])

In [43]:
xiH_b['xi']

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.69410843e-06,  8.76044135e-07, -1.87142708e-08,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.84715033e-06,  2.54140237e-06,  2.02308283e-06])